In [10]:
p = 0x31337313373133731337313373133731337313373133731337313373133732ad
a = 0xdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeef
b = 0xdeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0de

# s1 = a * s0 + b
# s2 = a * s1 + b
# s2 = a * (a * s0 + b) + b
# s2 = a^2 * s0 + a * b + b
# s3 = a * s2 + b
# s3 = a * (a^2 * s0 + a * b + b) + b
# s3 = a^3 * s0 + a^2 * b + a * b + b

# si = a^i * s0 + b * (a^(i - 1) + a^(i - 2) + ... + a^0)
# si = a^i * s0 + b * (a^i - 1) / (a - 1)

# h1 = s1 * s2^-1 (mod p) -> s1 - h1 * s2 = 0 (mod p)
# h2 = s3 * s4^-1 (mod p) -> s3 - h2 * s4 = 0 (mod p)

# s1 - h1 * s2 = 0 (mod p)
# (a * s0 + b) - h1 * (a^2 * s0 + a * b + b) = 0 (mod p)
# a * s0 + b - h1 * a^2 * s0 - h1 * a * b - h1 * b = 0 (mod p)
# a * s0 - h1 * a^2 * s0 = - b + h1 * a * b + h1 * b (mod p)
# s0 * (a - h1 * a^2) = - b + h1 * a * b + h1 * b (mod p)
# s0 = (- b + h1 * a * b + h1 * b) / (a - h1 * a^2) (mod p)

import itertools

def small_roots(f, bounds, m=1, d=None):
	if not d:
		d = f.degree()

	if isinstance(f, Polynomial):
		x, = polygens(f.base_ring(), f.variable_name(), 1)
		f = f(x)

	R = f.base_ring()
	N = R.cardinality()
	
	# f /= f.coefficients().pop(0)
	f = f.change_ring(ZZ)

	G = Sequence([], f.parent())
	for i in range(m+1):
		base = N^(m-i) * f^i
		for shifts in itertools.product(range(d), repeat=f.nvariables()):
			g = base * prod(map(power, f.variables(), shifts))
			G.append(g)

	B, monomials = G.coefficient_matrix()
	monomials = vector(monomials)

	factors = [monomial(*bounds) for monomial in monomials]
	for i, factor in enumerate(factors):
		B.rescale_col(i, factor)

	B = B.dense_matrix().LLL()

	B = B.change_ring(QQ)
	for i, factor in enumerate(factors):
		B.rescale_col(i, 1/factor)

	H = Sequence([], f.parent().change_ring(QQ))
	for h in filter(None, B*monomials):
		H.append(h)
		I = H.ideal()
		if I.dimension() == -1:
			H.pop()
		elif I.dimension() == 0:
			roots = []
			for root in I.variety(ring=ZZ):
				root = tuple(R(root[var]) for var in f.variables())
				roots.append(root)
			return roots

	return []


In [11]:
from Crypto.Util.number import long_to_bytes

def lcg(x, a, b):
    while True:
        yield (x := a*x + b)

trunc = 48
hint1 = 68043888471472326216028808313418042452773439804630775380245509
hint2 = 20393380408458084390734542085792035416934775783234285328508786

# Method #1 of for symbolic variables
# P1 = PolynomialRing(GF(p), names=("x", "y1", "y2"))
# x, y1, y2 = P1._first_ngens(3)

# Method #2 for symbolic variables
P2 = GF(p)
x, y1, y2 = polygens(P2, "x, y1, y2")

# Initializing LCG and the equation
gen = lcg(x, a, b)
f1 = next(gen) - ((hint1 * 2^trunc) + y1)*next(gen)
f2 = next(gen) - ((hint2 * 2^trunc) + y2)*next(gen)
print("f1 =", f1)
print("f2 =", f2)

# Method #1 of eliminating the 'x' variable
# I = P1.ideal([f1, f2])
# # for eq in I.groebner_basis():
# #     print("eq =", eq)
# eq = I.groebner_basis()[2]
# print("eq =", eq)

# Method #2 of eliminating the 'x' variable
R2 = P2["y1, y2"]
eq = R2(str(f1.sylvester_matrix(f2, x).det()))
print("eq =", eq)
print(type(eq))

# Method #1 of calculating the value of y1, y2 via small_roots()
# load("/tmp/challenges/bagon/coppersmith.sage")
# PP = PolynomialRing(GF(p), names=("y1", "y2"))
# y1, y2 = PP._first_ngens(2)
# eq = y1*y2 + 17958614578291060127372567165625104635968578406665148011793100393317766384261*y1 + 21482764867760921829316264896543186885960483717026930516358257269269010706591*y2 + 11461354639087106092183304818848118155588978531258855709017557388913692277944
# y1, y2 = small_roots(eq, [2^trunc, 2^trunc])[0]

# Method #2 of calculating the value of y1, y2 via small_roots()
# load("/tmp/challenges/bagon/coppersmith.sage")
y1, y2 = small_roots(eq, [2^trunc, 2^trunc])[0]

# Print the recovered h1 and h2
h1 = hint1 * 2^trunc + y1 
h2 = hint2 * 2^trunc + y2 
print("y1 =", y1)
print("y2 =", y2)
print("h1 =", h1)
print("h2 =", h2)

# Method #1 of calculating the original state (flag)
s0 = (- b + h1 * a * b + h1 * b) * pow(a - h1 * a^2, -1, p) % p
print("s0 =", s0)
print(long_to_bytes(Integer(s0)))

# Method #2 of calculating the original state (flag)
s0 = f1(y1=y1).univariate_polynomial().roots()[0][0]
print("s0 =", s0)
print(long_to_bytes(Integer(s0)))

f1 = 4694833141127635196669010981624964714934061881126774253066839868864162114864*x*y1 + 7099011241840247812343042630831396609149859034129515350071740257574603117641*x + 13841707355630389978850397639603099411364552931959812236734889864032072732340*y1 + 12654932858111333771871308639093018975473820049286015852580294994412022340444
f2 = 11845362698228032843594100271155616928826416609628434849155060432136898469371*x*y2 + 11247380007748480762447971098927370882122005186158296338740518294684461786029*x + 17286613946104093486820715914561001876676718597756038050197834092882387247655*y2 + 2312188405148099814112423541131832184096681543944638664527007754368564459689
eq = 18809326409291330862604976668985876478440966155687736658634682104375929937722*y1*y2 + 11041424088962944640659812143517142272736130456909284771510909223488989604487*y1 + 14233060980671932796431957443255525863085321076394449506003200931901787491501*y2 + 16746613822117990690889085600665851549680923440506018881296848205900133337620
<c

/tmp/ipykernel_625/601352598.py:49: DeprecationWarning: the function coefficient_matrix is deprecated; use coefficients_monomials instead
See https://github.com/sagemath/sage/issues/37035 for details.
  B, monomials = G.coefficient_matrix()


In [ ]:
from Crypto.Util.number import long_to_bytes

p = 0x31337313373133731337313373133731337313373133731337313373133732ad
a = 0xdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeefdeadbeef
b = 0xdeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0dedeadc0de

res1 = 552476625038229923263134487902261274257296153348743417699048
res2 = 15646013724991982279822443899088099194134757156140951422250765

def lcg(x, a, b):
    while True:
        yield (x := a*x + b)

P1.<x,y1,y2> = Zmod(p)[]
gen = lcg(x, a, b)

# h1 = s1 * s2^-1
# h2 = s3 * s4^-1

# h1 * s2 = s1
# h2 * s4 = s3

# h1 * s2 - s1 = 0 (mod p)
# h2 * s4 - s3 = 0 (mod p)

trunc = 48
f1 = - next(gen) + (res1 * 2^48 + y1) * next(gen)
f2 = - next(gen) + (res2 * 2^48 + y2) * next(gen)
print("f1 =", f1)
print("f2 =", f2)

# Method #1: Resultant
P2.<y1,y2> = GF(p)[]
f3 = P2(f1.sylvester_matrix(f2, x).det())
print("f3 =", f3)

# Method #2: Groebner Basis
# I = ideal([f1, f2])
# # for eq in I.groebner_basis():
# #     print("f3 =", eq)
# f3 = I.groebner_basis()[2]
# # <class 'sage.rings.polynomial.multi_polynomial_libsingular.MPolynomial_libsingular'>

# P1.<y1,y2> = Zmod(p)[]
# f3 = f3 = y1*y2 + 16622349651625494821084748728680622619580140010454328863313497382049570212485*y1 + 2485621290116592112588888747850621492970657477970199617823775554033656918175*y2 + 3927391354501905078866543920387680867278303491721004414159699627270920298240
# print("f3 =", f3)

load("/tmp/challenges/bagon/coppersmith.sage")
y1, y2 = small_roots(f3, [2^trunc, 2^trunc])[0]
print("y1 =", y1)
print("y2 =", y2)

h1 = res1 * 2^trunc + y1
h2 = res2 * 2^trunc + y2

# s0 = (- b + h1 * a * b + h1 * b) / (a - h1 * a^2) (mod p)
s0 = (- b + h1 * a * b + h1 * b) * pow(a - h1 * a^2, -1, p)
print("s0 =", s0, long_to_bytes(Integer(s0)))

f1 = 17559399858637399173906283962318814228819069940357188219030786464362082338173*x*y1 + 5393261658035753581648137519712980098006698404079348135290151441387218754195*x + 8412525644134644391724897304340679532388578889524150235362736469194171720697*y1 + 20512242006566079513967080822325112514248730844251337818225494073376440547420
f2 = 10408870301537001526981194672788162014926715211855527622942565901089345983666*x*y2 + 2060423309473200881591372981470074541549305715369815726225727221705507235514*x + 4967619053660940883754579029382777067076413223727924421899792240343857205382*y2 + 4172624818527538318928745905828167923124275183108882265814256094503809248775
f3 = 18809326409291330862604976668985876478440966155687736658634682104375929937722*y1*y2 + 199243323522563550074866355336470561247935118425411437737727589074960902172*y1 + 16601353654672170456729510220453012777005260543908659086702034127806773091189*y2 + 18415559606244325081044863679519912109666426315546062445141673232916979664254
y1 = 2

<string>:25: DeprecationWarning: the function coefficient_matrix is deprecated; use coefficients_monomials instead
See https://github.com/sagemath/sage/issues/37035 for details.
